# Análise de dataset

Determinando se tomate possui resistência ou não

## Integrantes

Daiane Estenio dos Santos, 84198\
Jacó Magalhães I Sen Chen, 84717\
João Victor Pessoa Queiroz, 85568\
Luís Fernando Pedro Paulino da Silva, 85398

In [5]:
import math
import re

In [1]:
# Math
def distancia_euclidiana(p1, p2):
    total = 0

    for i in range(len(p1)):
        total += (p1[i] - p2[i]) ** 2
    
    return math.sqrt(total)

def escala_normalizada(x, v_max, v_min):
    return (x - v_min) / (v_max - v_min)

In [11]:
# Util
def ler_arquivo(filename, keys):
    amostras = []
    descarte = []
    total_descarte = 0

    # Lê o arquivo e faz a normalização de acordo com a configuração
    with open(filename, "r") as dataset:
        for instancia in dataset.readlines():
            x = instancia.replace("\n", "").split(",")
            
            try:
                amostra_normalizada = normalizar_arquivo(x, keys)
                amostras.append(amostra_normalizada)
            except ValueError:
                total_descarte += 1
                descarte.append(x)

    # Gera o arquivo de saída para visualização dos dados normalizados
    escrever_arquivo("output.data", amostras)

    # Gera o arquivo de amostras descartadas
    escrever_arquivo("fix.data", descarte)
    print(f"Total amostras descartadas: {total_descarte}")

    return amostras

def escrever_arquivo(nome_arquivo, lista_dados):
    with open(nome_arquivo, "w") as output:
        for item in lista_dados:
            item_string = str(item).replace("[","").replace("]","")
            output.write(f"{item_string}\n")

def normalizar_arquivo(amostra, names):
    amostra_normalizada = []

    for indice in range(len(amostra)):
        # Obtém o dicionário da chave em questão
        itens = names[indice]
        v_min = 0
        v_max = 1
        decimal = 2
        normalize = 1
        valor = 0

        if "remove" in itens:
            continue
        # se possui decimal, usa ele invés do padrão
        if "decimal" in itens:
            decimal = itens["decimal"]
        # se estipulado um mínimo, altera para ele invés do padrão
        if "vmin" in itens:
            v_min = arred(float(re.sub("[^\d\.]+", "", str(itens["vmin"]))), decimal)
        # se estipulado um máximo, altera para ele
        if "vmax" in itens:
            v_max = arred(float(re.sub("[^\d\.]+", "", str(itens["vmax"]))), decimal)
        # verifica se deve fazer a normalização ou não
        if "normalize" in itens:
            normalize = itens["normalize"]

        # Posição atual no vetor da amostra
        valor_atual = amostra[indice]

        # regex verifica tudo o que não é número ou ponto e remove
        # Limita a quantidade de casas decimais
        valor = arred(float(re.sub("[^\d\.]+", "", str(valor_atual))), decimal)

        if normalize:
            valor = escala_normalizada(valor, v_max, v_min)
        
        amostra_normalizada.append(arred(float(valor), decimal))
    
    return amostra_normalizada

def arred(valor, decimal = None):
    if decimal is None:
        return valor
    
    if decimal == 0:
        return round(valor)
    
    return round(valor, decimal)


In [7]:
# Análise
def info_dataset(amostras, classe, info=True):
    output1, output2 = 0,0

    for amostra in amostras:
        if amostra[classe] == 1:
            output1 += 1 # Tomate tem resistência
        else:
            output2 += 1 # Tomate não tem resistência

    if info == True:
        print(f"Total de amostras................: {len(amostras)}")
        print(f"Total Normal (Sem recorrência)...: {output1}")
        print(f"Total Alterado (Com recorrência).:  {output2}")

    return [len(amostras), output1, output2]

def separar_amostras(amostras, porcentagem, classe):
    _, output1, output2 = info_dataset(amostras, classe)

    treinamento = []
    teste = []

    max_output1 = int(porcentagem*output1)
    max_output2 = int((1 - porcentagem)*output2)

    total_output1 = 0
    total_output2 = 0

    for amostra in amostras:
        if(total_output1 + total_output2) < (max_output1 + max_output2):
            # Inserir em treinamento
            treinamento.append(amostra)
            if amostra[classe] == 1 and total_output1 < max_output1:
                total_output1 += 1
            else:
                total_output2 += 1
        else:
            # Insere em teste
            teste.append(amostra)

    return [treinamento, teste]

def knn(treinamento, nova_amostra, classe, k):
    distancias = {}
    tamanho_treino = len(treinamento)

    # Calcula distância euclidiana
    for i in range(tamanho_treino):
        d = distancia_euclidiana(treinamento[i], nova_amostra)
        distancias[i] = d

    # Obtém k-vizinhos
    k_vizinhos = sorted(distancias, key=distancias.get)[:k] # retorna do começo até o k-1

    # Votação
    qtd_output1 = 0
    qtd_output2 = 0
    for indice in k_vizinhos:
        if treinamento[indice][classe] == 1: # saída normal
            qtd_output1 += 1
        else:                                # saída alterada
            qtd_output2 += 1
    
    if qtd_output1 > qtd_output2:
        return 1
    else:
        return 0

In [3]:
# Definição
names = {
    # id
    0:{
        "remove": 1,
    },
    # trat
    1:{
        "vmin": 11,
        "vmax": 334,
    },
    # NDVI_d28
    2:{
        "vmin": 0.649084685,
        "vmax": 0.82786736,
        "normalize": 0,
    },
    # SAVI_d28
    3:{
        "vmin": 0.973585561,
        "vmax": 1.241756375,
    },
    # GNDVI_d28
    4:{
        "vmin": 0.556921879,
        "vmax": 0.719607432,
        "normalize": 0,
    },
    # MCARI1_d28
    5:{
        "vmin": 11005.3558,
        "vmax": 19574.21117,
    },
    # SR_d28
    6:{
        "vmin": 4.789961492,
        "vmax": 11.79803884,
    },
    # NDVI_d01
    7:{
        "vmin": 0.635929577,
        "vmax": 0.803978099,
        "normalize": 0,
    },
    # SAVI_d01
    8:{
        "vmin": 0.953868658,
        "vmax": 1.205943923,
    },
    # GNDVI_d01
    9:{
        "vmin": 0.592643702,
        "vmax": 0.738357136,
    },
    # MCARI1_d01
    10:{
        "vmin": 18122.40482,
        "vmax": 36639.93661,
    },
    # SR_d01
    11:{
        "vmin": 4.520550195,
        "vmax": 9.869641031,
    },
    # NDVI_d04
    12:{
        "vmin": 0.642946343,
        "vmax": 0.790564939,
    },
    # SAVI_d04
    13:{
        "vmin": 0.964380991,
        "vmax": 1.185811722,
    },
    # GNDVI_d04
    14:{
        "vmin": 0.552339957,
        "vmax": 0.716055763,
    },
    # MCARI1_d04
    15:{
        "vmin": 9431.856566,
        "vmax": 23387.23253,
    },
    # SR_d04
    16:{
        "vmin": 4.653616843,
        "vmax": 8.930666224,
    },
    # NDVI_d08
    17:{
        "vmin": 0.655962402,
        "vmax": 0.816573783,
    },
    # SAVI_d08
    18:{
        "vmin": 0.983907832,
        "vmax": 1.224826448,
    },
    # GNDVI_d08
    19:{
        "vmin": 0.582567164,
        "vmax": 0.728203577,
    },
    # MCARI1_d08
    20:{
        "vmin": 11463.91334,
        "vmax": 23276.95995,
    },
    # SR_d08
    21:{
        "vmin": 4.902358902,
        "vmax": 10.37776304,
    },
    # Classe
    22:{
        "vmin": 0,
        "vmax": 1,
        "decimal": 0,
    }
}

In [17]:
# Teste
acertos = 0
# Posição da classe deve apontar para o arquivo normalizado
# Como é removido 1 coluna do arquivo final, isso também
# deve ser contabilizado
# (de acordo com a posição removida em relação a posição da classe)
pos_classe = 21
porcentagem = 0.7
k = 11
amostras = ler_arquivo("dataset_tomate.data", names)

treinamento, teste = separar_amostras(amostras, porcentagem, pos_classe)

for amostra in teste:
    classe_retornada = knn(treinamento, amostra, pos_classe, k)

    if amostra[pos_classe] == classe_retornada:
        acertos += 1

print(f"Total de treinamento..: {len(treinamento)}")
print(f"Total de testes.......: {len(teste)}")
print(f"Total de acertos......: {acertos}")
print(f"Porcentagem de acerto.: {arred(100*acertos/len(teste), 0)} %")

Total amostras descartadas: 1
Total de amostras................: 132
Total Normal (Sem recorrência)...: 91
Total Alterado (Com recorrência).:  41
Total de treinamento..: 75
Total de testes.......: 57
Total de acertos......: 53
Porcentagem de acerto.: 93 %
